- **TODO:** finish structuring train, test, main
- **TODO:** plot_accuracies_losses

In [1]:
import os
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from torch.utils.data.sampler import SubsetRandomSampler
from pathlib import Path
from PIL import Image

import warnings
warnings.filterwarnings("ignore")

In [18]:
# main.py

# root_dir = "datasets/office/amazon/images"
# path = "datasets/office/amazon"
# name_dataset = "office"

data_domain = "amazon"
img_size = 224
#
# Pytorch ImageFolder fits our dataset structure
# dataset = datasets.ImageFolder(root=root_dir, transform=data_transform)
image_data_folder = datasets.ImageFolder(root= "datasets/office/%s/images" % data_domain)
print("total no. of images in amazon dataset:",len(image_data_folder))

data_transforms = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])


total no. of images in amazon dataset: 2817


In [19]:
# dataloader.py
class OfficeAmazonDataset(Dataset):
    """Class to create an iterable dataset 
    of images and corresponding labels """
        
    def __init__(self, image_folder_dataset, transform=None):
        super(OfficeAmazonDataset, self).__init__()
        self.image_folder_dataset = image_folder_dataset
        self.transform = transform

    def __len__(self):
        return len(self.image_folder_dataset.imgs)
    
    def __getitem__(self, idx):
        # read image, class from folder_dataset given index
        img, img_label = image_folder_dataset[idx][0], image_folder_dataset[idx][1]
        
        # apply transformations (it already returns them as torch tensors)
        if self.transform is not None:
            self.transform(img)
        
        img_label_pair = {"image": img,
                         "class": img_label}
        
        return img_label_pair

def get_dataloader(dataset, batch_size):
        
    def get_subset(indices, start, end):
        return indices[start:start + end]
    
    # Split train/val data ratios
    TRAIN_RATIO, VALIDATION_RATIO = 0.7, 0.3
    train_set_size = int(len(dataset) * TRAIN_RATIO)
    validation_set_size = int(len(dataset) * VALIDATION_RATIO)

    # Generate random indices for train and val sets
    indices = torch.randperm(len(dataset))
    train_indices = get_subset(indices, 0, train_set_size)
    validation_indices = get_subset(indices, train_set_size, validation_set_size)
    # test_indices = get_subset(indices, train_count + validation_count, len(dataset))

    # Create sampler objects 
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(validation_indices)

    # Create data loaders for data
    train_loader = DataLoader(dataset, batch_size=batch_size, 
                              sampler=train_sampler, num_workers=4)

    val_loader = DataLoader(dataset, batch_size=batch_size, 
                            sampler=val_sampler, num_workers=4)
    
    return train_loader, val_loader

In [20]:
amazon_dataset = OfficeAmazonDataset(image_folder_dataset=image_data_folder, transform=data_transforms)
train_loader, val_loader = get_dataloader(amazon_dataset, batch_size=4)

### mean and std of dataset

In [10]:
import torch
from torchvision import transforms, datasets
import numpy as np


def compute_mean_std(path_dataset):
    """
    Compute mean and standard deviation of an image dataset.
    Acknowledgment : http://forums.fast.ai/t/image-normalization-in-pytorch/7534
    """
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor()
    ])

    dataset = datasets.ImageFolder(root=path_dataset,
                                   transform=transform)
    # Choose a large batch size to better approximate. Optimally load the dataset entirely on memory.
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=4096, shuffle=False, num_workers=4)

    pop_mean = []
    pop_std = []

    for i, data in enumerate(data_loader, 0):
        # shape (batch_size, 3, height, width)
        numpy_image = data[0].numpy()

        # shape (3,) -> 3 channels
        batch_mean = np.mean(numpy_image, axis=(0, 2, 3))
        batch_std = np.std(numpy_image, axis=(0, 2, 3))

        pop_mean.append(batch_mean)
        pop_std.append(batch_std)

    # shape (num_iterations, 3) -> (mean across 0th axis) -> shape (3,)
    pop_mean = np.array(pop_mean).mean(axis=0)
    pop_std = np.array(pop_std).mean(axis=0)

    values = {
        'mean': pop_mean,
        'std': pop_std
    }

    return values

In [ ]:
# retrieved from: https://forums.fast.ai/t/image-normalization-in-pytorch/7534/7
# this cell takes too much memory and time to compute
data_domain = "amazon"
path_dataset = "datasets/office/%s/images" % data_domain

transform = transforms.Compose([
        transforms.Resize((224, 224)), # original image size 300x300 pixels
        transforms.ToTensor()])

dataset = datasets.ImageFolder(root=path_dataset,
                               transform=transform)

# set large batch size to get good approximate of mean, std of full dataset
data_loader = DataLoader(dataset, batch_size=2048, shuffle=False, num_workers=0) # 4096

mean = []
std = []

# for i, data in enumerate(data_loader, 0):
    # shape is (batch_size, channels, height, width)
    npy_image = data[0].numpy()
    
    # compute mean, std per batch shape (3,) three channels
    batch_mean = np.mean(npy_image, axis=(0,2,3))
    batch_std = np.std(npy_image, axis=(0,2,3))
    
    mean.append(batch_mean)
    std.append(batch_std)
    
# shape (num_iterations, 3) -> (mean across 0th axis) -> shape (3,)
mean = np.array(mean).mean(axis=0) # average over batch averages
std = np.arry(std).mean(axis=0) # average over batch stds

values = {
    "mean": mean,
    "std": std
}

### testing dataloader for office dataset

In [14]:
from dataloader import get_office_dataloader
amazon_dataloader = get_office_dataloader("amazon", 2048)
amazon_dataloader.batch_size

2048

## Putting all together

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import torch
import torch.nn as nn
# from torch.autograd import Variable, Function
# from utils import load_state_dict_from_url
# from loss import CORAL_loss


class DeepCORAL(nn.Module):
	"""
	DeepCORAL network as defined in the paper.
	Network architecture based on following repository:
    https://github.com/SSARCandy/DeepCORAL/blob/master/models.py
    :param num_classes: int --> office dataset has 31 different classes
	"""
	def __init__(self, num_classes=1000):
		super(DeepCORAL, self).__init__()
		self.sharedNetwork = AlexNet()
		self.fc8 = nn.Linear(4096, num_classes) # fc8 activation

		# initiliaze fc8 weights according to the CORAL paper (N(0, 0.005))
		self.fc8.weight.data.normal_(0.0, 0.005)

	def forward(self, source, target): # computes activations for BOTH domains
		source = self.sharedNetwork(source)
		source = self.fc8(source)

		target = self.sharedNetwork(target)
		target = self.fc8(target)

		return source, target


class AlexNet(nn.Module):
	"""
	AlexNet model obtained from official Pytorch repository:
    https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py
	"""
	def __init__(self, num_classes=1000):
		super(AlexNet, self).__init__()

		self.features = nn.Sequential(
			nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding=2),
			nn.ReLU(inplace=True),
			nn.MaxPool2d(kernel_size=3, stride=2),
			nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, padding=2),
			nn.ReLU(inplace=True),
			nn.MaxPool2d(kernel_size=3, stride=2),
			nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, padding=1),
			nn.ReLU(inplace=True),
			nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1),
			nn.ReLU(inplace=True),
			nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
			nn.ReLU(inplace=True),
			nn.MaxPool2d(kernel_size=3, stride=2),
		)

		self.avgpool = nn.AdaptiveAvgPool2d(output_size=(6,6))

		self.classifier = nn.Sequential(
			nn.Dropout(),
			nn.Linear(256 * 6 * 6, 4096),
			nn.ReLU(inplace=True),
			nn.Dropout(),
			nn.Linear(4096, 4096),
			nn.ReLU(inplace=True), # take fc8 (without activation)
			# nn.Linear(4096, num_classes),
			)

	def forward(self, x):
		# define forward pass of network
		x = self.features(x)
		x = self.avgpool(x)
		x = torch.flatten(x, 1) # flatten to input into classifier
		# x = x.view(x.size(0), 246 * 6 * 6)
		x = self.classifier(x)
		return x


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from skimage import io
import matplotlib.pyplot as plt
import os
import torch
from torchvision import transforms, datasets
# from torch.utils import model_zoo
import numpy as np
try: # import pytorch data getters
	from torch.hub import load_state_dict_from_url
except ImportError:
	from torch.utils.model_zoo import load_url as load_state_dict_from_url


"""
Created on Saturday Feb 22 2020

@authors: Alan Preciado, Santosh Muthireddy
"""
def load_pretrained_AlexNet(model, progress=True):
# def alexnet(pretrained=False, progress=True, **kwargs):
	"""
	AlexNet model architecture from the paper
	pretrained (bool): If True, returns a model pre-trained on ImageNet
	progress (bool): If True, displays a progress bar of the download to stderr
	"""

	__all_ = ["AlexNet", "alexnet", "Alexnet"]

	model_url = {
	    'alexnet':'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
	}

	state_dict = load_state_dict_from_url(model_url['alexnet'], progress=progress)
	model_dict = model.state_dict() # check this one

	# filter out unmatching dictionary
	# reference: https://github.com/SSARCandy/DeepCORAL/blob/master/main.py
	state_dict = {k: v for k, v in state_dict.items() if k in model_dict}

	model_dict.update(state_dict)
	model.load_state_dict(state_dict)


def show_image(dataset, domain, image_class, image_name):
	"""
	Plot images from given domain, class
	"""
	image_file = io.imread(os.path.join("data", dataset, domain, "images", image_class, image_name))
	plt.imshow(image_file)
	plt.pause(0.001)
	plt.figure()


def accuracy(prediction, target):
	pass


def save_model(model, path):
	torch.save(model.state_dict(), path)
	print("checkpoint saved in {}".format(path))


def load_model(model, path):
	model.load_state_dict(torch.load(path))
	print("checkpoint loaded from {}".format(path))


def get_mean_std_dataset(root_dir):
	"""
	Function to compute mean and std of image dataset.
	Move batch_size param according to memory resources.
	retrieved from: https://forums.fast.ai/t/image-normalization-in-pytorch/7534/7
	"""

	# data_domain = "amazon"
	# path_dataset = "datasets/office/%s/images" % data_domain
    
	transform = transforms.Compose([
        transforms.Resize((224, 224)), # original image size 300x300 pixels
        transforms.ToTensor()])

	dataset = datasets.ImageFolder(root=root_dir,
	                               transform=transform)

	# set large batch size to get good approximate of mean, std of full dataset
	# batch_size: 4096, 2048
	data_loader = DataLoader(dataset, batch_size=2048,
	                        shuffle=False, num_workers=0)

	mean = []
	std = []

	for i, data in enumerate(data_loader, 0):
		# shape is (batch_size, channels, height, width)
		npy_image = data[0].numpy()

		# compute mean, std per batch shape (3,) three channels
		batch_mean = np.mean(npy_image, axis=(0,2,3))
		batch_std = np.std(npy_image, axis=(0,2,3))

		mean.append(batch_mean)
		std.append(batch_std)

	# shape (num_iterations, 3) -> (mean across 0th axis) -> shape (3,)
	mean = np.array(mean).mean(axis=0) # average over batch averages
	std = np.arry(std).mean(axis=0) # average over batch stds

	values = {
		"mean": mean,
		"std": std
	}

	return values


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import numpy as np
import os
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
from utils import get_mean_std_dataset


"""
Created on Saturday Feb 22 2020

@authors: Alan Preciado, Santosh Muthireddy
"""
class OfficeAmazonDataset(Dataset):
    """Class to create an iterable dataset
    of images and corresponding labels """

    def __init__(self, image_folder_dataset, transform=None):
        super(OfficeAmazonDataset, self).__init__()
        self.image_folder_dataset = image_folder_dataset
        self.transform = transform

    def __len__(self):
        return len(self.image_folder_dataset.imgs)

    def __getitem__(self, idx):
        # read image, class from folder_dataset given index
        img, img_label = image_folder_dataset[idx][0], image_folder_dataset[idx][1]

        # apply transformations (it already returns them as torch tensors)
        if self.transform is not None:
            self.transform(img)

        img_label_pair = {"image": img,
                         "class": img_label}

        return img_label_pair


def get_dataloader(dataset, batch_size, train_ratio=0.7):
    """
    Splits a dataset into train and test.
    Returns train_loader and test_loader.
    """

    def get_subset(indices, start, end):
        return indices[start:start+end]

    # Split train/val data ratios
    TRAIN_RATIO, VALIDATION_RATIO = train_ratio, 1-train_ratio
    train_set_size = int(len(dataset) * TRAIN_RATIO)
    validation_set_size = int(len(dataset) * VALIDATION_RATIO)

    # Generate random indices for train and val sets
    indices = torch.randperm(len(dataset))
    train_indices = get_subset(indices, 0, train_set_size)
    validation_indices = get_subset(indices, train_set_size, validation_set_size)
    # test_indices = get_subset(indices,train_count+validation_count,len(dataset))

    # Create sampler objects
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(validation_indices)

    # Create data loaders
    train_loader = DataLoader(dataset, batch_size=batch_size,
                              sampler=train_sampler, num_workers=4)

    val_loader = DataLoader(dataset, batch_size=batch_size,
                            sampler=val_sampler, num_workers=4)

    return train_loader, val_loader


def get_office_dataloader(name_dataset, batch_size, train=True):
    """
    Creates dataloader for the datasets in office datasetself.
    Makes use of get_mean_std_dataset to compute mean and std along the
    color channels for each dataset in office.
    """
    # Ideally compute mean and std with get_mean_std_dataset.py
    # Values retrieved from:
    # https://github.com/DenisDsh/PyTorch-Deep-CORAL/blob/master/data_loader.py

    root_dir = "datasets/office/%s/images" % name_dataset

    __datasets__ = ["amazon", "dslr", "webcam"]

    if name_dataset not in __datasets__:
        raise ValueError("must introduce one of the three datasets in office")

    mean_std = {
        "amazon":{
            "mean":[0.7923, 0.7862, 0.7841],
            "std":[0.3149, 0.3174, 0.3193]
        },
        "dslr":{
            "mean":[0.4708, 0.4486, 0.4063],
            "std":[0.2039, 0.1920, 0.1996]
        },
        "webcam":{
            "mean":[0.6119, 0.6187, 0.6173],
            "std":[0.2506, 0.2555, 0.2577]
        }
    }

    data_transforms = transforms.Compose([
            transforms.Resize((224, 224)),
            # transforms.RandomSizedCrop(224),
            # transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean_std[name_dataset]["mean"],
                                 std=mean_std[name_dataset]["std"])
        ])

    dataset = datasets.ImageFolder(root=root_dir,
                                   transform=data_transforms)

    # note on shuffling in data loader:
    # https://stackoverflow.com/questions/54354465/impact-of-using-data-shuffling-in-pytorch-dataloader
    dataset_loader = DataLoader(dataset,
                                batch_size=batch_size,
                                shuffle=train,
                                num_workers=4)

    return dataset_loader

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import torch


"""
Created on Saturday Feb 25 2020

@authors: Alan Preciado, Santosh Muthireddy
"""
def CORAL_loss(source, target):
	"""
	From the paper, the vectors that compose Ds and Dt are D-dimensional vectors.
	:param source: torch tensor: source data (Ds) with dimensions DxNs
	:param target: torch tensor: target data (Dt) with dimensons DxNt
	"""

	d = source.size(1) # d-dimensional vectors (same for source, target)

	source_covariance = compute_covariance(source)
	target_covariance = compute_covariance(target)

	# take Frobenius norm (https://pytorch.org/docs/stable/torch.html)
	loss = torch.norm(torch.mul((source_covariance-target_covariance),(source_covariance-target_covariance)), p="fro")
	# loss = torch.norm(torch.mm((source_covariance-target_covariance),(source_covariance-target_covariance)), p="fro")

	loss = loss / (4*d*d)

	return loss


def compute_covariance(data):
	"""
	Compute covariance matrix for given dataset as shown in paper.
	Equations 2 and 3.
	:param data: torch tensor: input source/target data
	"""

	# data dimensions: nxd (this for Ns or Nt)
	n = data.size(0)

	# proper matrix multiplication for right side of equation (2)
	ones_vector = torch.ones(n).resize(1, n) 	# 1xN dimensional vector (transposed)
	one_onto_D = torch.mm(ones_vector, data)
	mult_right_terms = torch.mm(one_onto_D.t(), one_onto_D)
	mult_right_terms = torch.div(mult_right_terms, n) # element-wise divison

	# matrix multiplication for left side of equation (2)
	mult_left_terms = torch.mm(data.t(), data)

	covariance_matrix = 1/(n-1) * torch.add(mult_left_terms, -1*(mult_right_terms))

	return covariance_matrix


In [1]:
# train.py
def train(model, optimizer, epoch, _lambda):
    """
    This method fits the network params one epoch at a time.
    Implementation based on: https://github.com/SSARCandy/DeepCORAL/blob/master/main.py
    """
    
    model.train()
    
    results = [] # list to append loss values at each epoch
    
    # first cast into an iterable list the data loaders
    # shapes: data_source -> (batch_size, channels, height, width), data_target -> (batch_size)
    # source[0][1][0].size() --> torch.Size([128, 3, 224, 224])

    source, target = list(enumerate(source_loader)), list(enumerate(target_loader))
    train_steps = min(len(source), len(target))
    
    # start batch training
    for batch_idx in tnrange(train_steps):
    # for batch_idx in range(train_steps):
        # second fetch data in batches
        # _, source_data -> torch.Size([128, 3, 224, 224]), labels -> torch.Size([128])
        _, (source_data, source_label) = source[batch_idx]
        _, (target_data, _) = target[batch_idx] # unsupervised learning
        
        if CUDA:
            # move to device
            source_data = source_data.cuda()
            source_label = source_label.cuda()
            target_data = target_data.cuda()
        
        # create pytorch variables, the variables and functions build a dynamic graph of computation
        source_data, source_label = Variable(source_data), Variable(source_label)
        target_data = Variable(target_data)
        
        # reset to zero optimizer gradients
        optimizer.zero_grad()
        
        # do a forward pass through network (recall DeepCORAL outputs source, target activation maps)
        output1, output2 = model(source_data, target_data)
        
        # compute losses (classification and coral loss)
        classification_loss = torch.nn.functional.cross_entropy(out1, source_label) # only for source data
        coral_loss = CORAL_loss(output1, output2)
        
        # compute total loss (equation 6 paper)
        total_loss = classification_loss + _lambda*coral_loss
        
        # compute gradients of network (backprop in pytorch)
        total_loss.backwards()
        
        # update weights of network
        optimizer.step()
        
        # append results for each batch iteration as dictionaries
        results.append({
            'epoch': epoch,
            'step': batch_idx + 1,
            'total_steps': train_steps,
            'lambda': _lambda,
            'coral_loss': coral_loss.data[0],
            'classification_loss': classification_loss.data[0],
            'total_loss': total_loss.data[0]
        })
        
        # print training info
        print('Train Epoch: {:2d} [{:2d}/{:2d}]\t'
              'Lambda value: {:.4f}, Classification loss: {:.6f}, CORAL loss: {:.6f}, Total_Loss: {:.6f}'.format(
                  epoch,
                  batch_idx + 1,
                  train_steps,
                  _lambda,
                  classification_loss.data[0],
                  coral_loss.data[0],
                  total_loss.data[0]
              ))

    return results

# test.py
def test(model, data_loader, epoch):
    """
    Computes classification accuracy of (labeled) data using cross-entropy.
    Retreived from: https://github.com/SSARCandy/DeepCORAL/blob/master/main.py
    """
    # eval() it indicates the model that nothing new is 
    # to be learnt and the model is used for testing
    model.eval()
    
    test_loss = 0
    correct_class = 0
    
    # go over dataloader batches, labels
    for data, label in data_loader:
        if CUDA:
            data, label = data.cuda(), label.cuda()
        
        # note on volatile: https://stackoverflow.com/questions/49837638/what-is-volatile-variable-in-pytorch
        data, label = Variable(data, volatile=True), Variable(label)
        output, _ = model(data, data) # just use one ouput of DeepCORAL
        
        # sum batch loss when computing classification
        test_loss += torch.nn.functional.cross_entropy(output, label, size_average=False).data[0]
        
        # get the index of the max log-probability
        pred = out.data.max(1, keepdim=True)[1]
        correct_class += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    
    # compute test loss as correclty classified labels divided by total data size
    test_loss = test_loss/len(data_loader.dataset)
    
    # return dictionary containing info of each epoch
    return {
        "epoch": epoch,
        "average_loss": test_loss,
        "correct_class": correct_class,
        "total elems": len(data_loader.dataset),
        "accuracy %": 100.*correct_class/len(data_loader.dataset)
    }

In [2]:
# main.py
from __future__ import division
import argparse
import torch
from torch.autograd import Variable
from loss import CORAL_loss
import tqdm
from tqdm import tnrange
from utils import load_pretrained_AlexNet, save_model, load_model
from dataloader import get_office_dataloader
from model import DeepCORAL, AlexNet

# define train parameters as in the paper (page 5)
CUDA = True if torch.cuda.is_available() else False
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 5e-4
MOMENTUM = 0.9
BATCH_SIZE = [32, 32] # batch_s, batch_t [128, 56]
EPOCHS = 1

# create dataloaders (Amazon as source and Webcam as target)
# TODO: try different combinations between 3 datasets to test adaptation
source_loader = get_office_dataloader(name_dataset="amazon", batch_size=BATCH_SIZE[0])
target_loader = get_office_dataloader(name_dataset="webcam", batch_size=BATCH_SIZE[1])

# argparse ...
model = DeepCORAL(num_classes=31) # input no. of classes in custom dataset

# define optimizer pytorch: https://pytorch.org/docs/stable/optim.html
# specify per-layer learning rates: 10*learning_rate for last two fc layers according to paper
optimizer = torch.optim.SGD([
    {"params": model.sharedNetwork.parameters()},
    {"params": model.fc8.parameters(), "lr":10*LEARNING_RATE},
], lr=LEARNING_RATE, momentum=MOMENTUM)

if CUDA:
    model = model.cuda()

In [3]:
# load pre-trained model --> TODO: check it is loading properly
# if args.load is not NONE:
    # load_model(model, args.load)
# else:
    # load_pretrained_AlexNet(model.sharedNetwork, progress=True)

load_pretrained_AlexNet(model.sharedNetwork, progress=True)

loading pre-trained model...
loaded model correctly...


In [ ]:
# store statistics of train/test
training_s_statistic = []
testing_s_statistic = []
testing_t_statistic = []

# iterate over epochs
for epoch in tnrange(EPOCHS):
    # compute lambda value
    _lambda = (epoch+1)/EPOCHS
    
    result_train = train(model, optimizer, epoch+1, _lambda)
    
    print('###EPOCH {}: Classification: {:.6f}, CORAL loss: {:.6f}, Total_Loss: {:.6f}'.format(
            epoch+1,
            sum(row['classification_loss'] / row['total_steps'] for row in result_train),
            sum(row['coral_loss'] / row['total_steps'] for row in result_train),
            sum(row['total_loss'] / row['total_steps'] for row in result_train),
        ))
    
    training_s_statistic.append(result_train)
    
    # check these ones
    test_source = test(model, source_loader, epoch)
    test_target = test(model, target_loader, epoch)
    testing_s_statistic.append(test_source)
    testing_t_statistic.append(test_target)
    
    print('###Test Source: Epoch: {}, avg_loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
            epoch+1,
            test_source['average_loss'],
            test_source['correct'],
            test_source['total'],
            test_source['accuracy'],
        ))
    
    print('###Test Target: Epoch: {}, avg_loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        epoch+1,
        test_target['average_loss'],
        test_target['correct'],
        test_target['total'],
        test_target['accuracy'],
    ))

In [1]:
def compute_covariance(data):
	"""
	Compute covariance matrix for given dataset as shown in paper (eqs 2 and 3).
	:param data: torch tensor: input source/target data
	"""

	# data dimensions: nxd (this for Ns or Nt)
	n = data.size(0) # get batch size

  # check gpu or cpu 
	if data.is_cuda:
		device = torch.device("gpu")
	else:
		device = torch.device("cpu")

	# proper matrix multiplication for right side of equation (2)
	ones_vector = torch.ones(n).resize(1, n).to(device=device) 	# 1xN dimensional vector (transposed)
	one_onto_D = torch.mm(ones_vector, data)
	mult_right_terms = torch.mm(one_onto_D.t(), one_onto_D)
	mult_right_terms = torch.div(mult_right_terms, n) # element-wise divison

	# matrix multiplication for left side of equation (2)
	mult_left_terms = torch.mm(data.t(), data)

	covariance_matrix= 1/(n-1) * torch.add(mult_left_terms,-1*(mult_right_terms))

	return covariance_matrix